In [114]:
import json
import pandas as pd
import glob
import datetime
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import os
import re
import shutil
import sys
import webbrowser
import math
import csv

In [115]:
#Contert time / input data
from decimal import Decimal
from datetime import datetime

def convert_datetime(a):
    date_formats = ["%Y-%m-%d %H:%M:%S.%f", 
                    "%Y-%m-%d %H:%M:%S", 
                    "%m/%d/%Y %H:%M",
                    "%m-%d-%Y %H:%M", 
                    "%m/%d/%Y", 
                    "%Y-%m-%d", 
                    "%H:%M.%f",
                    "%m/%d/%Y %H:%M:%S"] 
    for fmt in date_formats:
        try:
            return datetime.strptime(str(a), fmt).strftime("%Y-%m-%d")
        except ValueError:
            pass
    try:
        return (datetime(1899, 12, 30) + timedelta(days=float(a))).strftime("%Y-%m-%d")
    except ValueError:
        pass
    if not re.match("\d+:\d+:\d+(\.\d+)?", str(a)):
        return str(a)
    return  
    
def convert_date(a):
    my_format="%Y-%m-%d"
    try:
        return datetime.strptime(str(a), "%Y-%m-%d %H:%M:%S").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%m/%d/%Y %H:%M").strftime(my_format)
#alternation

def merge_csv_files(folder_path):
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

    dfs = []
    for file_path in file_paths:
        try:
            df = pd.read_csv(file_path, dtype={'task_id': object,'project_id':object,'object_id':object})
            df['task_id'] = "id=" + df['task_id'].astype(str)
            df['project_id'] = "id=" + df['project_id'].astype(str)
            df['object_id'] = "id=" + df['object_id'].astype(str)
            df['2_resolve_time'] = df['2_resolve_time'].apply(convert_datetime)
            
            df['source_file'] = os.path.basename(file_path)

            dfs.append(df)
        except Exception as e:
            print(f"Error reading file {file_path}: {str(e)}")
  
    merged_df = pd.concat(dfs, axis=0, ignore_index=True)

    return merged_df


In [116]:
# Supporting 

#Queue
queue_list = pd.read_excel('E:/tt/qa_daily_automation/Support_dashboard.xlsx',sheet_name='Queue List').values
queue_list = pd.DataFrame(queue_list)
headers = ['BUSINESS','COMPOUND','CATEGORY','MOD QUEUE ID','MOD QUEUE NAME',
           'QA QUEUE ID','QA QUEUE NAME','QUEUE GROUP','STATUS','Priority','Latency','AHT (secs)','Policy_type','Classify']
queue_list.columns = headers

#Week
week_list = pd.read_excel('E:/tt/qa_daily_automation/Support_dashboard.xlsx',sheet_name='Week')

#Variable
Date_variable = pd.read_excel('E:/tt/qa_daily_automation/variable.xlsx')
Date_variable['Start date'] = Date_variable['Start date'].apply(convert_datetime)
Date_variable['End date'] = Date_variable['End date'].apply(convert_datetime)
print(Date_variable)
Start_Date_False_Appeal = datetime.strptime(str(Date_variable.loc[0, 'Start date']), '%Y-%m-%d').strftime('%Y-%m-%d')
End_Date_False_Appeal = datetime.strptime(str(Date_variable.loc[0, 'End date']), '%Y-%m-%d').strftime('%Y-%m-%d')

Start_Date_DailyRP_Appeal = datetime.strptime(str(Date_variable.loc[1, 'Start date']), '%Y-%m-%d').strftime('%Y-%m-%d')
End_Date_DailyRP_Appeal = datetime.strptime(str(Date_variable.loc[1, 'End date']), '%Y-%m-%d').strftime('%Y-%m-%d')

            Unnamed: 0  Start date    End date
0     appeal_diff_case  2023-08-28  2023-08-28
1   appeal_dailyreport  2023-08-20  2023-08-27
2    regular_diff_case  2023-08-28  2023-08-28
3   regular_false_case  2023-08-27  2023-08-27
4     rock_dailyreport  2023-08-20  2023-08-27
5        PCR_diff_case  2023-08-28  2023-08-28
6       PCR_false_case  2023-08-27  2023-08-27
7      sideprj_shoptab  2023-08-28  2023-08-28
8        sideprj_ugads  2023-08-28  2023-08-28
9  sideprj_dailyreport  2023-08-20  2023-08-27


In [117]:
import datetime
try:
    def rename_files(data_dir):
        dates_set = set()

        for filename in pathlib.Path(data_dir).glob('*.csv'):
            if "Appeal" in filename.name:
                continue
            with open(filename, 'r') as csv_file:
                rows = csv.reader(csv_file, delimiter=',')
                headers = next(rows)
                sheet = {col: [] for col in headers}
                for row in rows:
                    for i, col in enumerate(headers):
                        sheet[col].append(row[i])
            if not sheet['2_resolve_time']:
                continue
            date_str = sheet['1_resolve_time'][1].split()[0]
            dates_set.add(date_str)

            title = sheet['title'][0].strip()

        for date in dates_set:
            start_date, end_date = date, date
            filenames = list(pathlib.Path(data_dir).glob('*.csv'))
            for filename in filenames:
                if "Appeal" in filename.name:
                    continue
                with open(filename, 'r') as csv_file:
                    rows = csv.reader(csv_file, delimiter=',')
                    headers = next(rows)
                    sheet = {col: [] for col in headers}
                    for row in rows:
                        for i, col in enumerate(headers):
                            sheet[col].append(row[i])
                if not sheet['2_resolve_time'] or len(sheet['2_resolve_time']) < 2:
                    continue
                date_list = []
                for date_str in sheet['2_resolve_time']:
                    date = None
                    if date_str:
                        date = datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S').date()
                    date_list.append(date)

                date_list = [d for d in date_list if d is not None]
                date_list.sort(reverse = True)
                if date_list:
                    start_date = date_list[-1].strftime('%Y-%m-%d')
                    end_date = date_list[0].strftime('%Y-%m-%d')
                else:
                    start_date, end_date = '1970-01-01', '1970-01-01'

                title = sheet['title'][0].strip()
                new_file_name = f"{title}_{start_date}_{end_date}.csv"
                new_filepath = os.path.join(data_dir, new_file_name)

                if os.path.exists(new_filepath):
                    os.remove(new_filepath)

                os.rename(filename, new_filepath)

    rename_files('D:/Appeal daily/')
except: pass

In [118]:
#Input rawdata
from datetime import datetime

qa_raw = merge_csv_files('E:/tt/qa_daily_automation/Appeal daily')
qa_raw['1_verifier'] = qa_raw['1_verifier'].str.replace('robot_estimate@','')
qa_raw = qa_raw[qa_raw['2_resolve_time'].notnull()]

In [119]:
qa_raw[qa_raw['project_id'] == 'id= 7.18178E+18']['source_file'].unique()

array([], dtype=object)

In [120]:
qa_raw['2_resolve_time_test'] = pd.to_datetime(qa_raw['2_resolve_time'])

In [121]:
qa_raw_test = qa_raw[(qa_raw['2_resolve_time_test'].dt.strftime('%Y-%m-%d') >= '2023-06-20') & (qa_raw['2_resolve_time_test'].dt.strftime('%Y-%m-%d') <= '2023-06-26') ]
qa_raw_test['title'].value_counts()

title
QA [VN]Live Slice Appeal Audit              3192
QA [VN]Anchor Video Appeal Audit New        2394
QA [VN]Shop Product Appeal Audit            2394
QA [VN][Live Pirated]Appeal Audit           2394
QA [VN][Video Counterfeit]Appeal Audit      2295
QA [VN][Live Counterfeit]Appeal Audit       2238
QA [VN_CB]Shop Product Appeal Audit         1604
QA [VN][Product Counterfeit]Appeal Audit     867
Name: count, dtype: int64

In [122]:
keymap  = {
    "Not Approve": ['"status":400','":400,"','400.0','disapproval', 'disapproved','not approval','not approve' ,'notapproval','disapprove','not approved'],
    "Approve": ['approve','"status":200' ,'approval', 'approved','200' ,'200.0']
}

def check_string(s):
    if isinstance(s, str):
        s_lower = s.lower()
    
        for approval_status, substrings in keymap.items():
            if any(substring in s_lower for substring in substrings):
                return approval_status

    return s 

qa_raw['Mod Result'] = qa_raw['1_verify_data'].apply(check_string)
qa_raw['QA Result'] = qa_raw['2_verify_data'].apply(check_string)

In [123]:
qa_raw['Mod Result'].unique()

array(['Not Approve', 'Approve'], dtype=object)

In [124]:
### convert time
qa_raw['2_resolve_time'] =  qa_raw['2_resolve_time'].apply(convert_datetime)
qa_raw['1_resolve_time'] =  qa_raw['1_resolve_time'].apply(convert_datetime)


qa_raw['2_resolve_time'] = pd.to_datetime(qa_raw['2_resolve_time'])
qa_raw['1_resolve_time'] = pd.to_datetime(qa_raw['1_resolve_time'])

In [125]:
#Add diff case
qa_raw.loc[qa_raw['Mod Result'] != qa_raw['QA Result'],'Diff case']="FALSE"
qa_raw.loc[qa_raw['Diff case'].isnull(),'Diff case']="TRUE"

In [126]:
#Remove dup cases
qa_raw = qa_raw.drop_duplicates(subset='task_id',keep='first')

In [127]:
### cleaning

def filter_time( df,  from_date, to_date):

    appeal_rawdata = df[(df['2_resolve_time'].dt.strftime('%Y-%m-%d') >= from_date) & (df['2_resolve_time'].dt.strftime('%Y-%m-%d') <= to_date) ]
    filer_tcv = appeal_rawdata['1_verifier'].str.contains('@trans-cosmos.com.vn',na=False)
    appeal_rawdata = appeal_rawdata[filer_tcv].dropna(axis=0, subset = ['1_verifier'])
    appeal_rawdata = appeal_rawdata[['title','project_id', '1_verifier','1_resolve_time','object_id','Mod Result','1_reason','1_duration','2_verifier','2_resolve_time','task_id','tcs_link','QA Result','2_reason']]
    
    appeal_rawdata.to_excel(f"Appeal daily appeal task.xlsx", index=False)
   
    return appeal_rawdata

Final_appeal_rawdata = filter_time(qa_raw, Start_Date_False_Appeal,End_Date_False_Appeal)


# sample size and diff case


In [128]:
### sample size and diff case

def filter_time_2( df, from_date, to_date):
    
    qa_raw['1_resolve_time'] = pd.to_datetime(qa_raw['1_resolve_time'])
    appeal_rawdata_2 = df[(df['1_resolve_time'].dt.strftime('%Y-%m-%d') >= from_date) & (df['1_resolve_time'].dt.strftime('%Y-%m-%d') <= to_date) ]
    filer_tcv = appeal_rawdata_2['1_verifier'].str.contains('@trans-cosmos.com.vn',na=False)
    appeal_rawdata_2 = appeal_rawdata_2[filer_tcv].dropna(axis=0, subset = ['1_verifier'])
    appeal_rawdata_2 = appeal_rawdata_2[['title','project_id', '1_verifier','1_resolve_time','object_id','Mod Result','1_reason','1_duration','2_verifier','2_resolve_time','task_id','tcs_link','QA Result','2_reason']]
    return appeal_rawdata_2

Final_appeal_rawdata_2 = filter_time_2(qa_raw, Start_Date_DailyRP_Appeal,End_Date_DailyRP_Appeal)

In [129]:
Final_appeal_rawdata_2

,title,project_id,1_verifier,1_resolve_time,object_id,Mod Result,1_reason,1_duration,2_verifier,2_resolve_time,task_id,tcs_link,QA Result,2_reason
31861,QA [VN]Anchor Video Appeal Audit New,id=7181737555035996674,linh.vta@trans-cosmos.com.vn,2023-08-20,id=7269035983197323525,Not Approve,Appeal_reason_supporting_doc_insufficient,90,duc.vh@trans-cosmos.com.vn,2023-08-20,id=7269127428323213826,https://tcs-sg.bytelemon.com/worktable/7181737...,Not Approve,Appeal_reason_supporting_doc_insufficient
31862,QA [VN]Anchor Video Appeal Audit New,id=7181737555035996674,linh.vta@trans-cosmos.com.vn,2023-08-20,id=7268721161695953158,Not Approve,Appeal_reason_supporting_doc_insufficient,78,duc.vh@trans-cosmos.com.vn,2023-08-20,id=7269127448472633858,https://tcs-sg.bytelemon.com/worktable/7181737...,Not Approve,Appeal_reason_supporting_doc_insufficient
31863,QA [VN]Anchor Video Appeal Audit New,id=7181737555035996674,phuong.mtt@trans-cosmos.com.vn,2023-08-20,id=7268988031405703430,Approve,NaN,6,duc.vh@trans-cosmos.com.vn,2023-08-20,id=7269127461013586433,https://tcs-sg.bytelemon.com/worktable/7181737...,Approve,NaN
31864,QA [VN]Anchor Video Appeal Audit New,id=7181737555035996674,phuong.mtt@trans-cosmos.com.vn,2023-08-20,id=7269021122967470341,Not Approve,Appeal_reason_supporting_doc_insufficient,41,duc.vh@trans-cosmos.com.vn,2023-08-20,id=7269127475559498241,https://tcs-sg.bytelemon.com/worktable/7181737...,Not Approve,Appeal_reason_supporting_doc_insufficient
31865,QA [VN]Anchor Video Appeal Audit New,id=7181737555035996674,lap.na@trans-cosmos.com.vn,2023-08-20,id=7268891003665285382,Not Approve,Appeal_reason_supporting_doc_insufficient,24,duc.vh@trans-cosmos.com.vn,2023-08-20,id=7269127500096127489,https://tcs-sg.bytelemon.com/worktable/7181737...,Not Approve,Appeal_reason_supporting_doc_insufficient
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240902,QA [VN_CB]Shop Product Appeal Audit,id=7181778901155267073,gam.bth@trans-cosmos.com.vn,2023-08-27,id=7271555998102503685,Not Approve,NaN,99,nhat.pq@trans-cosmos.com.vn,2023-08-27,id=7271729733291737602,https://tcs-sg.bytelemon.com/worktable/7181778...,Not Approve,NaN
240903,QA [VN_CB]Shop Product Appeal Audit,id=7181778901155267073,gam.bth@trans-cosmos.com.vn,2023-08-27,id=7271508173859815686,Not Approve,NaN,91,nhat.pq@trans-cosmos.com.vn,2023-08-27,id=7271729742640890369,https://tcs-sg.bytelemon.com/worktable/7181778...,Not Approve,NaN
240904,QA [VN_CB]Shop Product Appeal Audit,id=7181778901155267073,hieu.tn@trans-cosmos.com.vn,2023-08-27,id=7271213995284021510,Not Approve,NaN,74,nhat.pq@trans-cosmos.com.vn,2023-08-27,id=7271729752795398657,https://tcs-sg.bytelemon.com/worktable/7181778...,Not Approve,NaN
240905,QA [VN_CB]Shop Product Appeal Audit,id=7181778901155267073,hieu.tn@trans-cosmos.com.vn,2023-08-27,id=7271208686314356997,Not Approve,NaN,45,nhat.pq@trans-cosmos.com.vn,2023-08-27,id=7271729762173796865,https://tcs-sg.bytelemon.com/worktable/7181778...,Not Approve,NaN


In [130]:
Final_appeal_rawdata_2['1_resolve_time'] = pd.to_datetime(Final_appeal_rawdata_2['1_resolve_time'].apply(convert_date))

Sample export - Appeal

In [131]:
Sample_size_appeal = Final_appeal_rawdata_2[['1_resolve_time','title','1_verifier']]
Sample_size_appeal = Final_appeal_rawdata_2.groupby(by=['1_resolve_time','title','1_verifier'],as_index=False).agg({'task_id':'count'})
Sample_size_appeal.to_excel(f"Appeal daily sample size.xlsx", index=False)
pv_Sample_size_appeal = pd.pivot_table(Sample_size_appeal,values='task_id',index=['title'],columns='1_resolve_time',aggfunc=np.sum)
pv_Sample_size_appeal

1_resolve_time,2023-08-20,2023-08-21,2023-08-22,2023-08-23,2023-08-24,2023-08-25,2023-08-26,2023-08-27
title,,,,,,,,
QA [VN]Anchor Video Appeal Audit New,114,114,114,114,114,114,114,114
QA [VN]Live Slice Appeal Audit,114,114,114,114,114,114,114,114
QA [VN]Shop Product Appeal Audit,114,113,114,114,114,114,114,114
QA [VN][Live Counterfeit]Appeal Audit,67,79,92,114,111,113,114,108
QA [VN][Live Pirated]Appeal Audit,114,114,114,114,114,114,114,114
QA [VN][Product Counterfeit]Appeal Audit,66,72,101,73,91,90,84,114
QA [VN][Video Counterfeit]Appeal Audit,114,114,114,114,114,114,114,114
QA [VN_CB]Shop Product Appeal Audit,59,23,114,100,78,74,98,94


Diff export - Appeal

In [132]:
#Diff case appeal

Final_appeal_rawdata_2.loc[Final_appeal_rawdata_2['Mod Result']!=Final_appeal_rawdata_2['QA Result'],'Diff case']="FALSE"
Final_appeal_rawdata_2.loc[Final_appeal_rawdata_2['Diff case'].isnull(),'Diff case']="TRUE"
False_appeal = Final_appeal_rawdata_2[Final_appeal_rawdata_2['Diff case']=="FALSE"]
False_appeal = False_appeal.groupby(by=['1_resolve_time','title','1_verifier'],as_index=False).agg({'Diff case':'count'})
pv_diff_appeal = pd.pivot_table(False_appeal,values='Diff case',index=['title'],columns='1_resolve_time',aggfunc=np.sum)
pv_diff_appeal

1_resolve_time,2023-08-20,2023-08-21,2023-08-22,2023-08-23,2023-08-24,2023-08-25,2023-08-26,2023-08-27
title,,,,,,,,
QA [VN]Anchor Video Appeal Audit New,1.0,NaN,1.0,2.0,1.0,1.0,1.0,NaN
QA [VN]Live Slice Appeal Audit,2.0,1.0,NaN,NaN,1.0,2.0,2.0,1.0
QA [VN]Shop Product Appeal Audit,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN
QA [VN][Live Counterfeit]Appeal Audit,NaN,1.0,2.0,1.0,1.0,NaN,1.0,2.0
QA [VN][Live Pirated]Appeal Audit,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0
QA [VN][Product Counterfeit]Appeal Audit,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN
QA [VN][Video Counterfeit]Appeal Audit,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN
QA [VN_CB]Shop Product Appeal Audit,NaN,NaN,1.0,4.0,1.0,NaN,1.0,NaN


In [133]:
False_appeal

,1_resolve_time,title,1_verifier,Diff case
0,2023-08-20,QA [VN]Anchor Video Appeal Audit New,phuong.mtt@trans-cosmos.com.vn,1
1,2023-08-20,QA [VN]Live Slice Appeal Audit,hanh.nt2@trans-cosmos.com.vn,1
2,2023-08-20,QA [VN]Live Slice Appeal Audit,hieu.ntm@trans-cosmos.com.vn,1
3,2023-08-20,QA [VN][Video Counterfeit]Appeal Audit,hau.nq@trans-cosmos.com.vn,1
4,2023-08-20,QA [VN][Video Counterfeit]Appeal Audit,phuong.mtt@trans-cosmos.com.vn,1
5,2023-08-21,QA [VN]Live Slice Appeal Audit,phung.pk1@trans-cosmos.com.vn,1
6,2023-08-21,QA [VN][Live Counterfeit]Appeal Audit,hanh.nt2@trans-cosmos.com.vn,1
7,2023-08-22,QA [VN]Anchor Video Appeal Audit New,thien.th@trans-cosmos.com.vn,1
8,2023-08-22,QA [VN][Live Counterfeit]Appeal Audit,duc.nh2@trans-cosmos.com.vn,1
9,2023-08-22,QA [VN][Live Counterfeit]Appeal Audit,hanh.nt2@trans-cosmos.com.vn,1


In [134]:
sum(False_appeal['Diff case'])

44

Daily Performance - Appeal

In [135]:
daily_performance_appeal = pd.merge(Sample_size_appeal, False_appeal,how='left',left_on=['1_resolve_time','title','1_verifier'],right_on=['1_resolve_time','title','1_verifier'])
daily_performance_appeal['Diff case'] = daily_performance_appeal['Diff case'].fillna(0)

daily_performance_appeal['task_id_1'] = daily_performance_appeal['task_id'].astype(str)
daily_performance_appeal['Diff_case_1'] = daily_performance_appeal['Diff case'].astype(int).astype(str)

daily_performance_appeal['%Accr'] = round(1-(daily_performance_appeal['Diff case'] / daily_performance_appeal['task_id']),4)


pv_daily_performance_appeal = pd.pivot_table(daily_performance_appeal,values=['Diff_case_1','task_id_1'],index=['title','1_verifier'],columns='1_resolve_time',aggfunc=lambda x: ' '.join(x))
pv_daily_performance_appeal = pv_daily_performance_appeal.droplevel(0,axis=1)

pv_daily_performance_appeal.sort_index(axis=1, level=0, ascending = True,inplace=True)
pv_daily_performance_appeal = daily_performance_appeal.drop(columns=['task_id_1','Diff_case_1'])
daily_performance_appeal = daily_performance_appeal[['1_resolve_time','title','1_verifier','task_id','Diff case']]
pv_daily_performance_appeal

,1_resolve_time,title,1_verifier,task_id,Diff case,%Accr
0,2023-08-20,QA [VN]Anchor Video Appeal Audit New,duyen.pmk@trans-cosmos.com.vn,21,0.0,1.0000
1,2023-08-20,QA [VN]Anchor Video Appeal Audit New,hau.nq@trans-cosmos.com.vn,4,0.0,1.0000
2,2023-08-20,QA [VN]Anchor Video Appeal Audit New,lap.na@trans-cosmos.com.vn,21,0.0,1.0000
3,2023-08-20,QA [VN]Anchor Video Appeal Audit New,linh.vta@trans-cosmos.com.vn,12,0.0,1.0000
4,2023-08-20,QA [VN]Anchor Video Appeal Audit New,phuong.mtt@trans-cosmos.com.vn,39,1.0,0.9744
...,...,...,...,...,...,...
420,2023-08-27,QA [VN][Video Counterfeit]Appeal Audit,van.ltt1@trans-cosmos.com.vn,7,0.0,1.0000
421,2023-08-27,QA [VN_CB]Shop Product Appeal Audit,gam.bth@trans-cosmos.com.vn,31,0.0,1.0000
422,2023-08-27,QA [VN_CB]Shop Product Appeal Audit,hieu.tn@trans-cosmos.com.vn,23,0.0,1.0000
423,2023-08-27,QA [VN_CB]Shop Product Appeal Audit,quyen.tt3@trans-cosmos.com.vn,30,0.0,1.0000


In [136]:
#Daily report

# daily_performance['%Accr'] = round((daily_performance['No. of Samples']-daily_performance['No. Diff cases'])//daily_performance['No. of Samples'],2)
daily_performance_appeal[['1_resolve_time','title','1_verifier','Diff case','task_id']]
daily_report_groupqueue = pd.merge(daily_performance_appeal,queue_list,how='left',left_on=['title'],right_on=['QA QUEUE NAME'])
daily_report_groupqueue = daily_report_groupqueue[['1_resolve_time','MOD QUEUE NAME','Diff case','task_id']].rename(columns={'task_id':'Sampling'})
daily_report_groupqueue = daily_report_groupqueue.groupby(['1_resolve_time','MOD QUEUE NAME'],as_index=False).agg({'Sampling':'sum','Diff case':'sum'})
daily_report_groupqueue['%Accr'] = round(1-(daily_report_groupqueue['Diff case'] / daily_report_groupqueue['Sampling']),4)
daily_report_groupqueue

,1_resolve_time,MOD QUEUE NAME,Sampling,Diff case,%Accr
0,2023-08-20,[VN]Anchor Video Appeal Audit New,114,1.0,0.9912
1,2023-08-20,[VN]Live Slice Appeal Audit,114,2.0,0.9825
2,2023-08-20,[VN]Shop Product Appeal Audit,114,0.0,1.0000
3,2023-08-20,[VN][Live Counterfeit]Appeal Audit,67,0.0,1.0000
4,2023-08-20,[VN][Live Pirated]Appeal Audit,114,0.0,1.0000
...,...,...,...,...,...
59,2023-08-27,[VN][Live Counterfeit]Appeal Audit,108,2.0,0.9815
60,2023-08-27,[VN][Live Pirated]Appeal Audit,114,1.0,0.9912
61,2023-08-27,[VN][Product Counterfeit]Appeal Audit,114,0.0,1.0000
62,2023-08-27,[VN][Video Counterfeit]Appeal Audit,114,0.0,1.0000


In [137]:
all_sheets_appeal = {'Diff_case_Appeal': Final_appeal_rawdata,
                     'Daily_Performance_Appeal':daily_performance_appeal,
                     'pv daily_performance_Appeal': pv_daily_performance_appeal,
                     'False_case_Appeal': False_appeal,
                     'Sample_size_Appeal': Sample_size_appeal,
                     'DR_group_queue_Appeal': daily_report_groupqueue}

In [138]:
# Appeal_writer = pd.ExcelWriter('C:/Users/v6210227/Desktop/appeal.xlsx', engine='xlsxwriter')

# for sheet_name in all_sheets_appeal.keys():
#     all_sheets_appeal[sheet_name].to_excel(Appeal_writer, sheet_name=sheet_name, index=True)

# Appeal_writer.close()